# CH08 텍스트 분석
- 텍스트 분류
- 감성분석
- 텍스트 요약
- 텍스트 군집화

## 1. 텍스트 분석 이해
-비정형 데이터인 텍스트 분석  
-피처 벡터화/피처 추출: 텍스트를 word 기반의 다수의 피처로 추출하고 단어 빈도수와 같은 숫자 값 부여  
-이 방법에는 Bag of Words와 Word2Vec가 있음  
  
-수행 프로세스  
- 텍스트 전처리: 미리 클렌징;대/소문자 변경, 특수문제 삭제 등의 토큰화 작업, 정규화 작업
-피처 벡터화/추출: 가공된 텍스트에서 피처를 추출하고 벡터값 할당  
  - BOW-Count 기반, TF-IDF 기반 벡터화
  - Word2Vec
-ML 모델 수립 및 학습/예측/평가  
  
-분석패키지
  - NLTK
  - Gensim: 토핑 모델링에 두각
  - SpaCy: 뛰어난 수행성능
  


## 2. 텍스트 전처리-텍스트 정규화
- 클렌징: 불필요한 문자, 기호 등 사전 제거
- 토큰화
  - 문장 토큰화: 문서에서 문장을 분리(보통 .  /n기호 이용)
  - 단어 토큰화: 단어를 토큰으로 분리
  - n-gram: 문맥적인 의미 무지되는 문제 해결하고자 도입
- Stopword 제거
- Stemming(단순)과 Lemmatization(더 정확,오래)


#### **토큰화**

In [ ]:
#문장 토큰화
from nltk import sent_tokenize
import nltk
nltk.download('punkt')

text_sample='The Matrix is everywhere its all around us, here even in this room.\You can see it out your window or on your television. \You feel it when you go to work, or go to church or pay your taxes.'
sentences=sent_tokenize(text=text_sample)
print(type(sentences),len(sentences))
print(sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
<class 'list'> 2
['The Matrix is everywhere its all around us, here even in this room.\\You can see it out your window or on your television.', '\\You feel it when you go to work, or go to church or pay your taxes.']


In [ ]:
#단어 토큰화
from nltk import word_tokenize
sentence="The Matrix is everywhere its all around us, here even in this room."
words=word_tokenize(sentence)
print(type(words),len(words))
print(words)

<class 'list'> 15
['The', 'Matrix', 'is', 'everywhere', 'its', 'all', 'around', 'us', ',', 'here', 'even', 'in', 'this', 'room', '.']


In [ ]:
from nltk import word_tokenize,sent_tokenize
def tokenize_text(text):
  sentences=sent_tokenize(text)
  word_tokens=[word_tokenize(sentence) for sentence in sentences]
  return word_tokens

word_tokens=tokenize_text(text_sample)
print(type(word_tokens),len(word_tokens))
print(word_tokens)

<class 'list'> 2
[['The', 'Matrix', 'is', 'everywhere', 'its', 'all', 'around', 'us', ',', 'here', 'even', 'in', 'this', 'room.\\You', 'can', 'see', 'it', 'out', 'your', 'window', 'or', 'on', 'your', 'television', '.'], ['\\You', 'feel', 'it', 'when', 'you', 'go', 'to', 'work', ',', 'or', 'go', 'to', 'church', 'or', 'pay', 'your', 'taxes', '.']]


#### **스톱 워드 제거**

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
print('영어 stop words 개수:',len(nltk.corpus.stopwords.words('english')))
print(nltk.corpus.stopwords.words('english')[:20])

영어 stop words 개수: 179
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his']


In [ ]:
#스탑 워드 처리 후 의미 있는 단어만 추출
import nltk

stopwords=nltk.corpus.stopwords.words('english')
all_tokens=[]

for sentence in word_tokens:
  filtered_words=[]
  for word in sentence:
    word=word.lower()
    if word not in stopwords:
      filtered_words.append(word)
  all_tokens.append(filtered_words)

print(all_tokens)

[['matrix', 'everywhere', 'around', 'us', ',', 'even', 'room.\\you', 'see', 'window', 'television', '.'], ['\\you', 'feel', 'go', 'work', ',', 'go', 'church', 'pay', 'taxes', '.']]


In [ ]:
#Stemming
from nltk.stem import LancasterStemmer
stemmer=LancasterStemmer()

print(stemmer.stem('working'),stemmer.stem('works'),stemmer.stem('worked'))
print(stemmer.stem('amusing'),stemmer.stem('amuses'),stemmer.stem('amused'))
print(stemmer.stem('happier'),stemmer.stem('happiest'))
print(stemmer.stem('fancier'),stemmer.stem('fanciest'))

work work work
amus amus amus
happy happiest
fant fanciest


In [ ]:
#Lemmatization
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')

lemma=WordNetLemmatizer()
print(lemma.lemmatize('amusing','v'),lemma.lemmatize('amuses','v'),lemma.lemmatize('amused','v'))
print(lemma.lemmatize('happier','a'),lemma.lemmatize('happiest','a'))
print()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
amuse amuse amuse
happy happy



## 03. Bag of Words
-문서가 가지는 모든 단어를 문맥이나 순서를 무시하고 일괄적으로 단어에 대해 빈도 값을 부여해 피처 값을 추출하는 모델  
-장점: 쉽고 빠른 구축  
-단점: 문맥 의미 반영 부족/희소 행렬 문제  

### BOW 피처 벡터화
-모든 문서에서 모든 단어를 칼럼 형태로 나열하고 각 문서에서 해당 단어의 횟수나 정규화된 빈도를 값으로 부여하는 데이터 세트 모델로 변경하는 것  
-카운트 기반의 벡터화 > TF-IDF 기반의 벡터화

### 사이킷런의 Count 및 TF-IDF 벡터화 구현: 
1. CountVectorizer
-피처 벡터화, 소문자 일괄 변환, 토큰화, 스톱 워드 필터링 등의 텍스트 전처리 수행  
-max_depth: 스톱 워드 같은 단어 제외 위함  
 min_df: 크게 중요하지 않거나 가비지성 단어 제외  
 max_features: 추출하는 피처의 수 제한  
 stop_words: 'english'로 지정하면 영어의 스톱 워드로 지정된 단어는 추출에서 제외  
 n_gram_range: 단어 순서를 어느 정도 보강하기 우한 범위 설정  
 analyzer: 피처 추출을 수행한 단위 지정  
 token_pattern: 토큰화를 수행하는 정규 표현식 패턴 지정  
 tokenizer: 별도의 커스텀 함수로 이용시 적용  
-사전 데이터 가공>토큰화>텍스트 정규화>피처 벡터화
 
2. TfidVectorizer


### 희소행렬
-너무 많은 불필요한 0값이 메모리 공간에 할당되어 메모리 공간 많이 필요
-행렬의 크기가 커서 연산 시에도 데이터 액세스를 위한 시간 많이 소모
-COO 형식, CSR 형식: 물리적으로 적은 메모리 공간 차지할 수 있도록 변환

### COO 형식
-0이 아닌 데이터만 별도의 array에 저장하고, 그 데이터가 가리키는 행과 열의 위치를 별도의 배열로 저장

In [ ]:
import numpy as np
dense=np.array([[3,0,1],[0,2,0]])

In [ ]:
from scipy import sparse
data=np.array([3,1,2])

row_pos=np.array([0,0,1])
col_pos=np.array([0,2,1])

sparse_coo=sparse.coo_matrix((data,(row_pos,col_pos)))

In [ ]:
sparse_coo=sparse.coo_matrix((data,(row_pos,col_pos)))

In [ ]:
sparse_coo.toarray()

array([[3, 0, 1],
       [0, 2, 0]])

### CSR 형식
-반복적인 위치 데이터를 사용해야 하는 문제점 해결  
-고유 값의 시작 위치만 알고 있으면 얼마든지 행 위치 배열을 다시 만들 수 있어 메모리가 적게 들고 빠른 연산 가능  
-csr_matrix 이용

In [ ]:
from scipy import sparse

dense2=np.array([[0,0,1,0,0,5],
                [1,4,0,3,2,5],
                [0,6,0,3,0,0],
                [2,0,0,0,0,0],
                [0,0,0,7,0,8],
                [1,0,0,0,0,0]])

data2=np.array([1,5,1,4,3,2,5,6,3,2,7,8,1])

row_pos=np.array([0,0,1,1,1,1,1,2,2,3,4,4,5])
col_pos=np.array([2,5,0,1,3,4,5,1,3,0,3,5,0])

sparse_coo=sparse.coo_matrix((data2,(row_pos,col_pos)))

row_pos_ind=np.array([0,2,7,9,10,12,13])

sparse_csr=sparse.csr_matrix((data2,col_pos,row_pos_ind))

print('C00 변환된 데이터가 제대로 되었는지 다시 Dense로 출력 확인')
print(sparse_coo.toarray())
print('CSR 변환된 데이터가 제대로 되었는지 다시 Dense로 출력 확인')
print(sparse_csr.toarray())

C00 변환된 데이터가 제대로 되었는지 다시 Dense로 출력 확인
[[0 0 1 0 0 5]
 [1 4 0 3 2 5]
 [0 6 0 3 0 0]
 [2 0 0 0 0 0]
 [0 0 0 7 0 8]
 [1 0 0 0 0 0]]
CSR 변환된 데이터가 제대로 되었는지 다시 Dense로 출력 확인
[[0 0 1 0 0 5]
 [1 4 0 3 2 5]
 [0 6 0 3 0 0]
 [2 0 0 0 0 0]
 [0 0 0 7 0 8]
 [1 0 0 0 0 0]]


In [ ]:
dense3=np.array([[0,0,1,0,0,5],
                [1,4,0,3,2,5],
                [0,6,0,3,0,0],
                [2,0,0,0,0,0],
                [0,0,0,7,0,8],
                [1,0,0,0,0,0]])

coo=sparse.coo_matrix(dense3)
csr=sparse.csr_matrix(dense3)

## 5. 감성 분석
-문서의 주관적인 감성/의견/감정 등을 파악하기 위한 방법  
-문서 내 텍스트가 나타내는 여러가지 주관적인 단어와 문맥을 기반으로 감성 수치 계산  
-지도학습: 학습 데이터와 타깃 레이블 값을 기반으로 감성 분석 학습을 수행한 뒤, 이를 기반으로 다른 데이터와의 감성 분석 예측  
 비지도학습: 'Lexicon'이라는 일종의 감성 어휘 사전 이용

### 지도학습 기반 감성 분석 실습-IMDB 영화평

In [ ]:
import pandas as pd

review_df=pd.read_csv("labeledTrainData.tsv",header=0,sep="\t",quoting=3)
review_df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


In [ ]:
print(review_df['review'][0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

정규 표현식 이용해 공백 변환

In [ ]:
import re

review_df['review']=review_df['review'].str.replace('<br \>',' ')
review_df['review']=review_df['review'].apply(lambda x : re.sub("[^a-zA-Z]"," ", x) )

<ipython-input-11-7d3ab98e5ee3>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  review_df['review']=review_df['review'].str.replace('<br \>',' ')


In [ ]:
from sklearn.model_selection import train_test_split

class_df=review_df['sentiment']
feature_df=review_df.drop(['id','sentiment'],axis=1,inplace=False)

X_train,X_test,y_train,y_test=train_test_split(feature_df,class_df,test_size=0.3,random_state=156)
X_train.shape,X_test.shape

((17500, 1), (7500, 1))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,roc_auc_score

In [ ]:
#Count 벡터화
pipeline=Pipeline([
    ('cut_vect',CountVectorizer(stop_words='english',ngram_range=(1,2) )),
    ('lr_clf',LogisticRegression(C=10))])

In [ ]:
pipeline.fit(X_train['review'],y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('cut_vect',
                 CountVectorizer(ngram_range=(1, 2), stop_words='english')),
                ('lr_clf', LogisticRegression(C=10))])

In [ ]:
pred=pipeline.predict(X_test['review'])

In [ ]:
pred_probs=pipeline.predict_proba(X_test['review'])[:,1]

In [ ]:
print('예측 정확도는 {0:.4f}, ROC-AUN는 {1:.4f}'.format(accuracy_score(y_test,pred),roc_auc_score(y_test,pred_probs)))

예측 정확도는 0.8865, ROC-AUN는 0.9506


In [ ]:
#TF-IDF 벡터화
pipeline=Pipeline([
    ('cut_vect',TfidfVectorizer(stop_words='english',ngram_range=(1,2) )),
    ('lr_clf',LogisticRegression(C=10))])

pipeline.fit(X_train['review'],y_train)
pred=pipeline.predict(X_test['review'])
pred_probs=pipeline.predict_proba(X_test['review'])[:,1]

print('예측 정확도는 {0:.4f}, ROC-AUN는 {1:.4f}'.format(accuracy_score(y_test,pred),roc_auc_score(y_test,pred_probs)))

예측 정확도는 0.8932, ROC-AUN는 0.9600


## 비지도학습 기반 감성 분석 소개
-감성 사전은 긍정/부정 감성의 정도를 의미하는 수치를 갖고 있으며 이를 감성 지수라고 함  
-감성 지수는 단어의 위치나 주변 단어, 문맥, POS 등을 참고해 결정 
-NLP 패키지의 WordNet: 시맨틱(문맥상 의미) 분석을 제공하는 어휘 사전  
    - 예측성능은 그리 좋지 않아 다른 감성 사진 많이 사용  
     - SentiWordNet(3가지 감성점수;긍정,부정,객관성 지수)  
     VADER: 주로 소셜 미디이ㅓ 텍스트에 이용  
     Pattern: 예측 성능 측면에서 가장 주목  



## SentiWordNet을 이용한 감성 분석

In [ ]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\judy0\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\judy0\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\judy0\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\judy0\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\judy0\AppData\Roaming\nltk_data...
[nltk_data]    | 

True

In [ ]:
from nltk.corpus import wordnet as wn
term='present'

synsets=wn.synsets(term)
print('synsets() 반환 type :',type(synsets))
print('synsets() 반환 값 개수 :',len(synsets))
print('synsets() 반환 값 :',synsets)

synsets() 반환 type : <class 'list'>
synsets() 반환 값 개수 : 18
synsets() 반환 값 : [Synset('present.n.01'), Synset('present.n.02'), Synset('present.n.03'), Synset('show.v.01'), Synset('present.v.02'), Synset('stage.v.01'), Synset('present.v.04'), Synset('present.v.05'), Synset('award.v.01'), Synset('give.v.08'), Synset('deliver.v.01'), Synset('introduce.v.01'), Synset('portray.v.04'), Synset('confront.v.03'), Synset('present.v.12'), Synset('salute.v.06'), Synset('present.a.01'), Synset('present.a.02')]


In [ ]:
for synset in synsets :
    print('##### Synset name : ',synset.name(),'#####')
    print('POS :',synset.lexname())
    print('Definition:',synset.definition())
    print('Lemmas:',synset.lemma_names())

##### Synset name :  present.n.01 #####
POS : noun.time
Definition: the period of time that is happening now; any continuous stretch of time including the moment of speech
Lemmas: ['present', 'nowadays']
##### Synset name :  present.n.02 #####
POS : noun.possession
Definition: something presented as a gift
Lemmas: ['present']
##### Synset name :  present.n.03 #####
POS : noun.communication
Definition: a verb tense that expresses actions or states at the time of speaking
Lemmas: ['present', 'present_tense']
##### Synset name :  show.v.01 #####
POS : verb.perception
Definition: give an exhibition of to an interested audience
Lemmas: ['show', 'demo', 'exhibit', 'present', 'demonstrate']
##### Synset name :  present.v.02 #####
POS : verb.communication
Definition: bring forward and present to the mind
Lemmas: ['present', 'represent', 'lay_out']
##### Synset name :  stage.v.01 #####
POS : verb.creation
Definition: perform (a play), especially on a stage
Lemmas: ['stage', 'present', 'represen

In [ ]:
#단어 상호 유사도 살펴보기
tree=wn.synset('tree.n.01')
lion=wn.synset('lion.n.01')
tiger=wn.synset('tiger.n.02')
cat=wn.synset('can.n.01')
dog=wn.synset('dog.n.01')

entities=[tree,lion,tiger,cat,dog]
similarities=[]
entity_names=[entity.name().split('.')[0] for entity in entities]

for entity in entities:
    similarity=[round(entity.path_similarity(compared_entity),2)
               for compared_entity in entities]
    similarities.append(similarity)
    
similarity_df=pd.DataFrame(similarities,columns=entity_names,index=entity_names)
similarity_df

,tree,lion,tiger,can,dog
tree,1.00,0.07,0.07,0.09,0.12
lion,0.07,1.00,0.33,0.06,0.17
tiger,0.07,0.33,1.00,0.06,0.17
can,0.09,0.06,0.06,1.00,0.10
dog,0.12,0.17,0.17,0.10,1.00


**senti_synset**

In [ ]:
import nltk
from nltk.corpus import sentiwordnet as swn

senti_synsets=list(swn.senti_synsets('slow'))
print('senti_synsets() 반환 type :',type(senti_synsets))
print('senti_synsets() 반환 값 개수 :',len(senti_synsets))
print('senti_synsets() 반환 값 :',senti_synsets)

senti_synsets() 반환 type : <class 'list'>
senti_synsets() 반환 값 개수 : 11
senti_synsets() 반환 값 : [SentiSynset('decelerate.v.01'), SentiSynset('slow.v.02'), SentiSynset('slow.v.03'), SentiSynset('slow.a.01'), SentiSynset('slow.a.02'), SentiSynset('dense.s.04'), SentiSynset('slow.a.04'), SentiSynset('boring.s.01'), SentiSynset('dull.s.08'), SentiSynset('slowly.r.01'), SentiSynset('behind.r.03')]


In [ ]:
import nltk
from nltk.corpus import sentiwordnet as swn

father=swn.senti_synset('father.n.01')

print('father 긍정감성 지수: ',father.pos_score())
print('father 부정감성 지수: ',father.neg_score())
print('father 객관성 지수: ',father.obj_score())
print('\n')

fabulous=swn.senti_synset('fabulous.a.01')

print('fabulous 긍정감성 지수: ',fabulous.pos_score())
print('fabulous 부정감성 지수: ',fabulous.neg_score())

father 긍정감성 지수:  0.0
father 부정감성 지수:  0.0
father 객관성 지수:  1.0


fabulous 긍정감성 지수:  0.875
fabulous 부정감성 지수:  0.125


1. WordNet 이용해 문서를 다시 단어로 토큰화한 뒤 어근 추출(Lemmatization)&품사 태깅(POS Tagging)

In [ ]:
#품사 태깅
from nltk.corpus import wordnet as wn

def penn_to_wn(tag):
    if tag.startswith('J'):
            return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB

2. SentiSynset 클래스를 생성하고 Polarity Score 합산하는 함수 생성

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

def swn_polarity(text):
    
    sentiment = 0.0
    tokens_count = 0

    lemmatizer = WordNetLemmatizer()
    raw_sentences = sent_tokenize(text)
    
    for raw_sentence in raw_sentences:
        
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
        for word, tag in tagged_sentence:

            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                continue
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
            
            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue
            
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            sentiment += (swn_synset.pos_score() - swn_synset.neg_score())
            tokens_count += 1
        
    if not tokens_count:
        return 0

    if sentiment >= 0:
        return 1
    
    return 0

In [ ]:
review_df['preds']=review_df['review'].apply( lambda x : swn_polarity(x))
y_target=review_df['sentiment'].values
preds=review_df['preds'].values

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score
from sklearn.metrics import recall_score, f1_score, roc_auc_score
import numpy as np

print(confusion_matrix(y_target, preds))
print('정확도:', np.round(accuracy_score(y_target, preds), 4))
print('정밀도:', np.round(precision_score(y_target, preds), 4))
print('재현율:', np.round(recall_score(y_target, preds), 4))

[[7649 4851]
 [3578 8922]]
정확도: 0.6628
정밀도: 0.6478
재현율: 0.7138


## VADER를 이용한 감성 분석
-소셜 미디어 감성 분석 용도로 만들어진 룰 기반의 Lexicon  
-SentimentIntensityAnalyzer 클래스 이용

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

senti_analyzer = SentimentIntensityAnalyzer()
senti_scores = senti_analyzer.polarity_scores(review_df['review'][0])
print(senti_scores)

{'neg': 0.13, 'neu': 0.744, 'pos': 0.126, 'compound': -0.8278}


-polarity_scores(): 딕셔너리 형태의 감성 점수 반환  
-compound: neg,neu,pos 잘 조합해 -1에서 1 사이의 감성 지수 표현  
-0.1 이상이면 긍정, 이하면 부정으로 보통 판단

In [ ]:
def vader_polarity(review, threshold=0.1):
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(review)

    agg_score = scores['compound']
    final_sentiment = 1 if agg_score >= threshold else 0
    return final_sentiment

review_df['vader_preds'] = review_df['review'].apply(lambda x: vader_polarity(x,0.1))
y_target = review_df['sentiment'].values
vader_preds = review_df['vader_preds'].values

print(confusion_matrix(y_target, vader_preds))
print('정확도:', np.round(accuracy_score(y_target, vader_preds), 4))
print('정밀도:', np.round(precision_score(y_target, vader_preds), 4))
print('재현율:', np.round(recall_score(y_target, vader_preds), 4))

[[ 6819  5681]
 [ 1936 10564]]
정확도: 0.6953
정밀도: 0.6503
재현율: 0.8451
